In [1]:
from nltk.tokenize import sent_tokenize
import nltk

nltk.download('punkt')

import torch
import pandas as pd
from tqdm.notebook import tqdm

from sklearn.model_selection import train_test_split
from transformers import AdamW, get_linear_schedule_with_warmup
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

from transformers import BertTokenizer
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

from transformers import BertForSequenceClassification
from transformers import BertModel, BertConfig
import random
import numpy as np
import operator

from transformers import AutoTokenizer, AutoModel
from transformers import BertForSequenceClassification, BertTokenizer, TextClassificationPipeline
import json
import glob
import os
import csv
import threading
import time
import multiprocessing
from multiprocessing import Process, Manager

[nltk_data] Downloading package punkt to
[nltk_data]     /home/kit/stud/ulvhi/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
2021-08-05 10:37:03.144107: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


In [2]:
from Pipeline import Pipeline
import Adapters
import Processors
import Sinks
from imrad_classification import BERTClassificationHandler
from imrad_classification import ClassificationHandler
from haystack_copy import ElasticsearchDocumentStore
from queue import *


[nltk_data] Downloading package punkt to
[nltk_data]     /home/kit/stud/ulvhi/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
model_path='/pfs/data5/home/kit/stud/ulvhi/ML_Modells/UnpaywallFinetuned_SciBERT002_epoch_100words_batches5.model'
document_store = ElasticsearchDocumentStore(host="aifb-ls3-vm1.aifb.kit.edu", username="", password="", index="document", duplicate_documents= 'overwrite') #update ElasticsearchSink documents = true
e_adapt = Adapters.ElasticsearchAdapter(document_store, batch_size=10, filters={'_status':["1"]})

/home/kit/stud/ulvhi/.local/lib/python3.8/site-packages/elasticsearch/connection/base.py:208: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.13/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


In [25]:
model_path='/pfs/data5/home/kit/stud/ulvhi/ML_Modells/UnpaywallFinetuned_SciBERT002_epoch_100words_batches5.model'
document_store = ElasticsearchDocumentStore(host="aifb-ls3-vm1.aifb.kit.edu", username="", password="", index="document", duplicate_documents= 'overwrite') #update ElasticsearchSink documents = true
e_adapt = Adapters.ElasticsearchAdapter(document_store, batch_size=10, filters={'_status':["4"]})
print(len(e_adapt))


6404824


In [17]:
update_pipeline = Pipeline(e_adapt, batch_size=100, cpus=1, max_runtime=12000)
#lockArray definieren
update_pipeline.add_processor(Processors.IMRaDClassification(model_path))
update_pipeline.add_sink(Sinks.ElasticsearchSink(document_store))
#update_pipeline.add_sink(Sinks.CSVMetadataSink("testcsv.csv",["arixive-id", "_split_id"])) #

print(update_pipeline)

Some weights of the model checkpoint at allenai/scibert_scivocab_cased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were

(ElasticsearchAdapter) ---> *IMRaDClassification*
===> |_ElasticsearchSink_|


In [18]:
len(e_adapt)


15457

In [15]:
update_pipeline.process()

Processing batches: [############################################################] 773/773 Processed documents: 77241 Output Documents: 77241


In [ ]:
#start:12.13  6424681
#5674181
#sever start 19:35
#241500 processed
#start 19:40
#ende 23:10

In [ ]:
{'text': 'For example, the sparse approximations introduce FORMULA (FORMULA ) inducing variables FORMULA to distillate the latent function values FORMULA through prior or posterior approximation\xa0, thus reducing the time complexity to FORMULA . Moreover, further complexity reduction can be achieved by exploiting the structured inducing points and the iterative methods through matrix-vector multiplies, see for example\xa0, . In contrast to the global sparse approximation, the complexity of GP can also be reduced through distributed computation\xa0, and local approximation\xa0, .', 'score': None, 'probability': None, 'question': None, 'meta': {'arixive-id': '2005.08467', 'mag_id': 3024325720, '_split_id': 2, '_status': 3, 'Section': 'intro', 'datasets': [], 'has_datasets': 'false', 'methods': [], 'has_methods': 'false', 'title': 'Deep Latent-Variable Kernel Learning'}, 'embedding': None, 'id': 'e24203c2effb66a5cd08658d3a687ea8'}
{'text': 'After collecting traffic stats for each pair of source and destination, they will be sorted in descending order for their total amounts of hops\xa0(number of hops\xa0FORMULA \xa0number of flits) and CS paths will be formed from the top of this sorted list while conflicting CS connections are simply discarded. Since this algorithm prioritizes frequently used source and destination pairs as CS path candidates, it is able to find good combinations of CS paths for given traffic patterns.', 'score': None, 'probability': None, 'question': None, 'meta': {'arixive-id': '2005.08478', 'mag_id': 3025011640, '_split_id': 32, '_status': 3, 'Section': 'method', 'datasets': [], 'has_datasets': 'false', 'methods': [], 'has_methods': 'false', 'title': 'Energy-Efficient On-Chip Networks through Profiled Hybrid Switching'}, 'embedding': None, 'id': '94ccbf462451fd229a2c7afff05352ef'}
{'text': 'Having computed all FORMULA node degrees to which it wants to connect, the new node then broadcasts a message with these degree values. Once the existing nodes in the network receive such a message, the nodes of the desired degrees respond to the new node to establish a connection to it. Then, the new node selects the first FORMULA of the nodes with desired degrees and connects to them.', 'score': None, 'probability': None, 'question': None, 'meta': {'arixive-id': '1303.6323', 'mag_id': 2011954835, '_split_id': 35, '_status': 3, 'Section': 'method', 'datasets': [], 'has_datasets': 'false', 'methods': [], 'has_methods': 'false', 'title': 'Constructing Limited Scale-Free Topologies Over Peer-to-Peer Networks'}, 'embedding': None, 'id': '109d6fcd626af42e9aaf37bded0ea7e0'}
{'text': 'Essentially, FORMULA trades off tangential velocity and force; the smaller its value, the more tangential forces are penalized. This is a projection to circle problem and two cases can be identified. If the solution lies inside the cone, the problem is an unconstrained quadratic one, and the solution is FORMULA Otherwise, the solution lies on the boundary FORMULA where FORMULA . For FORMULA the solution approaches the frictionless case, while for FORMULA energy dissipation is increased.', 'score': None, 'probability': None, 'question': None, 'meta': {'arixive-id': '2007.11261', 'mag_id': 3044967063, '_split_id': 23, '_status': 3, 'Section': 'discussion', 'datasets': [], 'has_datasets': 'false', 'methods': [], 'has_methods': 'false', 'title': 'Contact-Implicit Trajectory Optimization using an Analytically Solvable\n  Contact Model for Locomotion on Variable Ground'}, 'embedding': None, 'id': '8c7348b6e86afd8da429dc5938980bbd'}
{'text': 'In this paper, we adopt a two-layered GCN\xa0 where the first layer (denoted as FORMULA ) maps the raw node features FORMULA to the intermediate embedding space, and the second layer (denoted as FORMULA ) further maps the intermediate node embeddings FORMULA to the output space. FORMULA where FORMULA and FORMULA are task-dependent output function and loss function, respectively. Note that a node-anchor affinity matrix FORMULA serves as a weighted adjacency matrix of a bipartite graph FORMULA allowing only direct connections between nodes and anchors.', 'score': None, 'probability': None, 'question': None, 'meta': {'arixive-id': '2006.13009', 'mag_id': 3036106327, '_split_id': 22, '_status': 3, 'Section': 'method', 'datasets': [], 'has_datasets': 'false', 'methods': [], 'has_methods': 'false', 'title': 'Iterative Deep Graph Learning for Graph Neural Networks: Better and\n  Robust Node Embeddings'}, 'embedding': None, 'id': '4d100e4ed690897704568cde63fbb3d9'}
{'text': 'Our goal is to maximize the ELBO subject to some restrictions on FORMULA . Before describing the form we choose for FORMULA we first note that the second integral in equation REF is itself a lower bound on the marginal probability of the FORMULA th group of observations: FORMULA Thus, for any particular value of FORMULA we can maximize the global ELBO over FORMULA by minimizing the KL divergence between FORMULA and FORMULA . Algorithms Our algorithm for optimizing the ELBO from equation REF is summarized in algorithm REF .', 'score': None, 'probability': None, 'question': None, 'meta': {'arixive-id': '1404.4114', 'mag_id': 1795258949, '_split_id': 18, '_status': 3, 'Section': 'method', 'datasets': [], 'has_datasets': 'false', 'methods': [], 'has_methods': 'false', 'title': 'Structured Stochastic Variational Inference'}, 'embedding': None, 'id': '2a40b8acfada63a62f797af9fb1424a9'}
len(e_adapt)

In [ ]:
jsonDocument={'text': 'The performance difference after the first seven queries is statistically significant with a FORMULA level, which goes to FORMULA after ten queries. Conclusions This paper introduces CLEO, a preference elicitation algorithm that, unlike existing approaches, handles preference elicitation tasks defined over hybrid domains and with uncertain human feedback. A combinatorial formulation of the unknown DM utility function is adopted. CLEO consists of an incremental procedure, iteratively optimizing the learned approximation of DM utility function to generate candidate solutions and refining the approximation based on the human feedback received.', 'score': None, 'probability': None, 'question': None, 'meta': {'arixive-id': '1508.04261', 'mag_id': 2206671546, '_status': 0, '_split_id': 115}, 'embedding': None, 'id': '1540cf135faebf5dae7806863bf4ec7a'}

In [ ]:
model_path=""
model = BertForSequenceClassification.from_pretrained("allenai/scibert_scivocab_cased", num_labels=5, output_attentions=False, output_hidden_states=False)
model.load_state_dict(torch.load(model_path, map_location=torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')))        
classificationPipeline = TextClassificationPipeline(model=model, tokenizer=BertTokenizer.from_pretrained('allenai/scibert_scivocab_cased'), device=0)

        
def evaluateIMRAD(document):
    helperArray=document["text"]
    labelArray=classificationPipeline(helperArray)
    for i in range (0, len(labelArray)):
        label=labelArray[i]["label"]
        if (label=="LABEL_0"):
            label="intro"
                
        if (label=="LABEL_1"):
            label="related"
                
        if (label=="LABEL_2"):
            label="method"
                
        if (label=="LABEL_3"):
            label="results"
                
        if (label=="LABEL_4"):
            label="discussion"

            document["meta"]["Section"]=label
            document["meta"]["_status"]=1
            
        return document

In [ ]:
print(evaluateIMRAD(jsonDocument))


In [ ]:
 if (selected_device==0):
                labelArray=self._classificationPipeline0(document["text"])
                
            elif (selected_device==1):
                labelArray=self._classificationPipeline1(document["text"])
                
            elif (selected_device==2):
                labelArray=self._classificationPipeline2(document["text"])

            elif (selected_device==3):
                labelArray=self._classificationPipeline3(document["text"])
                
            label=labelArray[0]["label"]
            if (label=="LABEL_0"):
                label="intro"
                
            if (label=="LABEL_1"):
                label="related"
                
            if (label=="LABEL_2"):
                label="method"
                
            if (label=="LABEL_3"):
                label="results"
                
            if (label=="LABEL_4"):
                label="discussion"

            document["meta"]["Section"]=label
            #print(document["meta"])
        